# 🤖 RL Football Champions - GPU Training

This notebook trains the DQN agent using your **exact repository code** on Google Colab's free GPU.

## Instructions:
1. Go to **Runtime → Change runtime type → Select T4 GPU**
2. Run all cells (**Runtime → Run all**)
3. Training saves to Google Drive every 5,000 episodes
4. Download `weights-latest.json` when done

In [ ]:
# Step 1: Mount Google Drive for saving weights
from google.colab import drive
drive.mount('/content/drive')

import os
OUTPUT_DIR = '/content/drive/MyDrive/rl-football-weights'
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f'✅ Weights will be saved to: {OUTPUT_DIR}')

In [ ]:
# Step 2: Install Node.js 20
!apt-get update -qq
!apt-get install -qq nodejs npm > /dev/null
!npm install -g n > /dev/null 2>&1
!n 20 > /dev/null 2>&1
!hash -r
!/usr/local/bin/node --version

In [ ]:
# Step 3: Clone your exact repository
!rm -rf /content/rl-demo
!git clone https://github.com/aeermumcu/rl-football.git /content/rl-demo
%cd /content/rl-demo/trainer
!ls -la

In [ ]:
# Step 4: Switch to GPU TensorFlow (the ONLY change from local)
# This replaces @tensorflow/tfjs-node with tfjs-node-gpu for GPU acceleration

# Update package.json to use GPU version
!sed -i 's/@tensorflow\/tfjs-node/@tensorflow\/tfjs-node-gpu/g' package.json
!cat package.json

In [ ]:
# Step 5: Update dqn-agent.js to use GPU TensorFlow
!sed -i 's/tfjs-node/tfjs-node-gpu/g' dqn-agent.js
!head -5 dqn-agent.js

In [ ]:
# Step 6: Install dependencies
!npm install 2>&1 | tail -5

In [ ]:
# Step 7: Update train.js to save to Google Drive
# Only change: output directory

train_js_patch = '''
// Patch: Save to Google Drive instead of local weights folder
const COLAB_OUTPUT = '/content/drive/MyDrive/rl-football-weights';
'''

# Read, patch, and write train.js
with open('train.js', 'r') as f:
    content = f.read()

# Replace the output directory
content = content.replace(
    "const outputDir = path.join(__dirname, 'weights');",
    "const outputDir = '/content/drive/MyDrive/rl-football-weights';"
)

with open('train.js', 'w') as f:
    f.write(content)

print('✅ train.js patched to save to Google Drive')

In [ ]:
# Step 8: Verify setup
print('📁 Files in trainer directory:')
!ls -la *.js

print('\n📦 TensorFlow version:')
!/usr/local/bin/node -e "console.log(require('@tensorflow/tfjs-node-gpu').version)"

In [ ]:
# 🚀 Step 9: START TRAINING!
# This uses your EXACT code, just with GPU TensorFlow
# Target: 100,000 episodes
# Saves every 5,000 episodes to Google Drive

!/usr/local/bin/node --expose-gc train.js --episodes=100000 --save-every=5000

## 📥 Download Your Weights

After training completes:
1. Go to Google Drive
2. Navigate to `My Drive > rl-football-weights`
3. Download `weights-latest.json`
4. Load it in your browser app!

---

### What's Different from Local?
Only ONE thing: `@tensorflow/tfjs-node` → `@tensorflow/tfjs-node-gpu`

Everything else is **100% identical** to your local trainer code.